<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-27 10:23:03
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18


-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.87 L
Current:  1.29 C
-------------------
Today PnL: -30.10 K (-0.23%)
Current PnL: -17.13 L (-12.15%)
CY Booked + Current PnL: -12.84 L (-9.1%)
-------------------
Total profit:  3.65 L
Total loss:  -20.78 L
-------------------
Total Booked + Current PnL: 14.14 L (12.4%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.34%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.09 C
Est FTT PnL: 80.11 L (62.28%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.41 C (109.93%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.84,H-LC,50.00,100918.0,17115.0,6883.0,0.03,20.42,6.82,28.63,42.0,2.49,0.80,34.86,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,3.85,X-LC,47.87,154475.0,13666.0,11864.0,-1.23,9.70,7.68,18.13,17.0,1.15,1.22,21.97,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,87.23,98697.0,20335.0,13699.0,-0.72,25.95,13.88,43.43,71.0,1.48,0.78,32.01,XY24,NTT,MISC
28,GILLETTE,8109.44,10330.69,8.15,H-SC,90.43,159460.0,21600.0,16169.0,-0.10,15.67,10.14,27.39,118.0,1.34,1.26,38.21,X40,ATH,FMCG
56,PIDILITIND,2504.06,3576.00,-18.44,X-LC,22.34,90411.0,15289.0,16871.0,-1.03,20.35,18.66,42.81,11.0,0.91,0.71,13.39,X40,BTT,CHEMICALS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.0,69.15,X-SC,82.98,112948.0,-39043.0,113163.0,7.62,-25.69,100.19,48.77,59.0,-0.35,0.89,29.74,XR,NTT,JEWELLERY
88,VALIANTORG,512.64,838.0,-354.52,H-SC,98.94,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
55,PFIZER,4245.50,6318.3,4.71,H-SC,100.00,121095.0,23448.0,24231.0,1.59,24.01,20.01,48.82,138.0,0.97,0.96,34.53,X40,ATH,PHARMA
10,BANDHANBNK,214.25,400.0,-34.39,H-SC,40.43,195100.0,-53430.0,268906.0,1.55,-21.50,137.83,86.70,117.0,-0.20,1.54,29.07,XY24,NTT,BANKS
82,TITAGARH,1097.23,1548.0,-8.18,H-SC,94.68,161332.0,-30683.0,109561.0,1.22,-15.98,67.91,41.08,127.0,-0.28,1.27,36.55,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAMMAANCAP,170.35,326.00,-159.50,M-SC,10.64,72570.0,-29640.0,123028.0,-1.84,-29.00,169.53,91.37,183.0,-0.24,0.57,17.80,XY25,NTT,FINANCE
45,JPPOWER,18.73,26.20,-44.15,L-SC,28.72,113368.0,-28474.0,85049.0,-1.51,-20.07,75.02,39.88,236.0,-0.33,0.89,14.89,XY24,NTT,POWER
38,INDIAMART,2327.09,4911.36,-53.15,H-SC,43.62,121905.0,-1431.0,138399.0,-1.38,-1.16,113.53,111.05,108.0,-0.01,0.96,19.45,AR,ATH,MISC
7,ASIANTILES,75.41,137.00,5838.89,M-SC,95.74,65161.0,-28649.0,105268.0,-1.36,-30.54,161.55,81.67,203.0,-0.27,0.51,26.19,XR,NTT,CERAMICS
75,STARHEALTH,551.56,761.00,1.49,M-SC,88.30,166091.0,-34125.0,110152.0,-1.32,-17.04,66.32,37.97,194.0,-0.31,1.31,33.36,XY24,NTT,INSURANCE


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.78,X-LC,57.45,248422.0,17823.0,113305.0,-1.15,7.73,45.61,56.86,1.0,0.16,1.96,13.39,X40,BTT,IT
81,TCS,3841.06,4998.00,-16.16,X-LC,38.30,241328.0,-23705.0,103530.0,-1.12,-8.94,42.90,30.12,2.0,-0.23,1.90,8.23,X40,BTT,IT
50,LTIM,5564.16,7197.33,-5.40,X-LC,86.17,182398.0,-17912.0,76698.0,-0.88,-8.94,42.05,29.35,4.0,-0.23,1.44,26.01,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-9.47,X-LC,81.91,81975.0,-28197.0,117273.0,0.93,-25.59,143.06,80.85,5.0,-0.24,0.65,20.74,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-5.61,X-LC,12.77,212731.0,-4008.0,68670.0,-0.20,-1.85,32.28,29.84,6.0,-0.06,1.68,11.59,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,4.71,H-SC,100.00,121095.0,23448.0,24231.0,1.59,24.01,20.01,48.82,138.0,0.97,0.96,34.53,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,87.23,98697.0,20335.0,13699.0,-0.72,25.95,13.88,43.43,71.0,1.48,0.78,32.01,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.84,H-LC,50.00,100918.0,17115.0,6883.0,0.03,20.42,6.82,28.63,42.0,2.49,0.80,34.86,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-18.44,X-LC,22.34,90411.0,15289.0,16871.0,-1.03,20.35,18.66,42.81,11.0,0.91,0.71,13.39,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,38.95,M-SC,70.21,144238.0,6660.0,87754.0,-0.86,4.84,60.84,68.62,182.0,0.08,1.14,25.05,X40N,NTT,TEXTILES
28,GILLETTE,8109.44,10330.69,8.15,H-SC,90.43,159460.0,21600.0,16169.0,-0.10,15.67,10.14,27.39,118.0,1.34,1.26,38.21,X40,ATH,FMCG
57,POLYCAB,5005.95,7541.35,6.49,H-MC,61.70,101184.0,16083.0,27016.0,0.04,18.90,26.70,50.65,92.0,0.60,0.80,27.19,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,4.71,H-SC,100.00,121095.0,23448.0,24231.0,1.59,24.01,20.01,48.82,138.0,0.97,0.96,34.53,X40,ATH,PHARMA
17,CAMS,3643.00,5250.99,-0.89,H-SC,30.85,111521.0,9517.0,35508.0,0.13,9.33,31.84,44.14,111.0,0.27,0.88,27.09,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-0.89,H-SC,30.85,111521.0,9517.0,35508.0,0.13,9.33,31.84,44.14,111.0,0.27,0.88,27.09,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,19.15,132691.0,3539.0,83529.0,-0.61,2.74,62.95,67.42,155.0,0.04,1.05,29.05,XR,NTT,BANKS
93,WIPRO,243.46,420.00,-10.16,M-LC,45.74,153779.0,2834.0,106615.0,-0.88,1.88,69.33,72.51,69.0,0.03,1.21,15.78,XR,NTT,IT
33,HEROMOTOCO,4311.81,6039.03,-1.43,H-MC,77.66,150717.0,-196.0,60649.0,-1.22,-0.13,40.24,40.06,89.0,-0.00,1.19,23.28,AR,ATH,AUTO
38,INDIAMART,2327.09,4911.36,-53.15,H-SC,43.62,121905.0,-1431.0,138399.0,-1.38,-1.16,113.53,111.05,108.0,-0.01,0.96,19.45,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TRIDENT,37.35,48.00,67.61,M-SC,93.62,80448.0,-11806.0,38108.0,-0.18,-12.80,47.37,28.51,220.0,-0.31,0.63,37.66,XR,NTT,TEXTILES
52,MASFIN,326.60,399.50,-23.67,H-SC,60.64,86715.0,-11265.0,33134.0,-1.03,-11.50,38.21,22.32,130.0,-0.34,0.68,25.77,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,87.23,98697.0,20335.0,13699.0,-0.72,25.95,13.88,43.43,71.0,1.48,0.78,32.01,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.84,H-LC,50.00,100918.0,17115.0,6883.0,0.03,20.42,6.82,28.63,42.0,2.49,0.80,34.86,AR,ATH,INSURANCE
88,VALIANTORG,512.64,838.00,-354.52,H-SC,98.94,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,4.71,H-SC,100.00,121095.0,23448.0,24231.0,1.59,24.01,20.01,48.82,138.0,0.97,0.96,34.53,X40,ATH,PHARMA
88,VALIANTORG,512.64,838.00,-354.52,H-SC,98.94,108069.0,-25217.0,109809.0,4.99,-18.92,101.61,63.47,131.0,-0.23,0.85,81.15,XR,NTT,CHEMICALS
78,TANLA,917.30,1963.11,-39.41,H-SC,97.87,148953.0,-90462.0,363416.0,-0.11,-37.78,243.98,114.01,110.0,-0.25,1.18,36.61,AR,ATH,IT
18,CERA,8421.60,10420.45,-18.85,X-SC,96.81,83811.0,-25670.0,51653.0,-0.85,-23.45,61.63,23.73,64.0,-0.50,0.66,26.85,X40N,ATH,CERAMICS
7,ASIANTILES,75.41,137.00,5838.89,M-SC,95.74,65161.0,-28649.0,105268.0,-1.36,-30.54,161.55,81.67,203.0,-0.27,0.51,26.19,XR,NTT,CERAMICS


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.41
1,25,40.32
2,50,69.35


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.19
XY25    14.39
X40     14.23
X40N    13.67
AR      12.56
XR      10.35
X200     1.44
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.39
M-SC    17.99
X-LC    17.43
H-MC     8.68
M-LC     7.74
H-LC     7.29
X-MC     5.66
M-MC     5.63
X-SC     2.90
L-SC     2.29
L-LC     1.17
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.20,-0.57,32.11
IT,9.97,-18.51,83.29
BANKS,9.61,-15.14,68.07
MISC,6.52,-30.68,88.49
INSURANCE,6.03,1.74,27.56
FINANCE,5.93,-26.97,87.26
PAINTS,5.45,-19.03,54.26
ELECTRICAL,4.83,-4.81,56.99
AUTO,4.50,-9.52,57.36


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2891816.0,26
AR,1253888.0,13
X40N,1214632.0,17
XR,1136855.0,14
X40,754934.0,11
XY25,481233.0,10
SR,200756.0,2
X200,76698.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2306459.0,21
M-SC,2031423.0,21
X-LC,771192.0,12
M-MC,554916.0,5
H-MC,525282.0,8
X-SC,340586.0,4
M-LC,324303.0,6
X-MC,317568.0,5
L-SC,306962.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      926586.0      6
M-SC       XY24      751405.0      6
H-SC       AR        562150.0      3
           X40N      508871.0      6
M-SC       X40N      467768.0      6
X-LC       X40       433248.0      5
M-MC       XY24      390633.0      3
M-SC       XR        294327.0      4
H-SC       XR        268452.0      4
H-LC       AR        251117.0      4
M-SC       X40       216593.0      2
M-LC       XY24      198785.0      4
H-MC       XY24      185970.0      2
L-SC       XR        157417.0      2
X-MC       XY24      142349.0      2
X-LC       AR        141708.0      2
M-SC       XY25      123028.0      1
X-LC       XY25      119538.0      4
X-SC       SR        115468.0      1
H-MC       XR        113352.0      1
X-SC       XR        113163.0      1
           X40N      111955.0      2
M-LC       XR        106615.0      1
X-MC       X40N       99022.0      2
M-SC       AR         93014.0      1
           SR         85288.0      1
L-SC       XY24       85049.0      1
M-MC       XR         83529.0      1
           AR         80754.0      1
X-LC       X200       76698.0      1
X-MC       XY25       76197.0      1
H-MC       XY25       73602.0      1
           X40        64693.0      2
L-SC       AR         64496.0      1
H-MC       AR         60649.0      1
L-LC       XY25       41138.0      1
H-SC       X40        40400.0      2
H-LC       XY25       28827.0      1
H-MC       X40N       27016.0      1
M-LC       XY25       18903.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
